# Forecasting: Importación de librerías y carga de datos

En este notebook se importan las librerías necesarias y se carga el archivo de inferencia para el proceso de forecasting.

In [67]:
# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import holidays
import os
from datetime import datetime
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.inspection import permutation_importance
import joblib

## Cargar archivo de inferencia en DataFrame

A continuación se carga el archivo `ventas_2025_inferencia` desde la carpeta `data/raw/Inferencia` en un DataFrame llamado `Ventas_2025_inferencia`.

In [68]:
# Cargar archivo de inferencia en DataFrame
# Ajuste de path para Windows y posibles mayúsculas/minúsculas
ruta_archivo = os.path.join('..','data', 'raw', 'Inferencia', 'ventas_2025_inferencia.csv')
if not os.path.exists(ruta_archivo):
    ruta_archivo = os.path.join('data', 'raw', 'inferencia', 'ventas_2025_inferencia.csv')
if os.path.exists(ruta_archivo):
    inferencia_df = pd.read_csv(ruta_archivo)
    display(inferencia_df.head())
else:
    print(f"No se encontró el archivo en las rutas probadas.")

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [69]:
inferencia_df.shape

(888, 13)

## Preparación y transformación de inferencia_df

En esta sección se replican todas las transformaciones, ingeniería de variables y codificaciones realizadas sobre el dataframe df en el notebook de entrenamiento, para dejar inferencia_df listo para la inferencia con el modelo.

In [70]:
# Paso 1: Conversión de fecha a datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'], errors='coerce')
inferencia_df['producto_id'] = inferencia_df['producto_id'].astype(str)

print(f'✓ Fecha convertida a datetime')
print(f'✓ Forma del dataframe inicial: {inferencia_df.shape}')

✓ Fecha convertida a datetime
✓ Forma del dataframe inicial: (888, 13)


In [71]:
# Paso 2: Variables temporales y de calendario

# Variables básicas de fecha
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.weekday  # 0=Lunes, 6=Domingo

# Nombre del día
try:
    inferencia_df['nombre_dia'] = inferencia_df['fecha'].dt.day_name(locale='es_ES')
except:
    inferencia_df['nombre_dia'] = inferencia_df['fecha'].dt.day_name()

# Es fin de semana
inferencia_df['es_fin_de_semana'] = inferencia_df['dia_semana'].isin([5, 6]).astype(int)

# Festivos en España
festivos_es = holidays.country_holidays('ES', years=inferencia_df['año'].unique())
festivos_ts = set([pd.Timestamp(f) for f in festivos_es])
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(festivos_ts).astype(int)

# Black Friday (último viernes de noviembre)
def es_black_friday(fecha):
    if fecha.month == 11:
        ultimo_viernes = max(
            day for day in range(24, 31)
            if pd.Timestamp(year=fecha.year, month=11, day=day).weekday() == 4
        )
        return int(fecha.day == ultimo_viernes)
    return 0

inferencia_df['es_blackfriday'] = inferencia_df['fecha'].apply(es_black_friday)

# Cyber Monday (lunes siguiente a Black Friday)
def es_cyber_monday(fecha):
    if fecha.month == 11 or (fecha.month == 12 and fecha.day <= 3):
        ultimo_viernes = max(
            day for day in range(24, 31)
            if pd.Timestamp(year=fecha.year, month=11, day=day).weekday() == 4
        )
        black_friday = pd.Timestamp(year=fecha.year, month=11, day=ultimo_viernes)
        cyber_monday = black_friday + pd.Timedelta(days=3)
        return int(fecha == cyber_monday)
    return 0

inferencia_df['es_cybermonday'] = inferencia_df['fecha'].apply(es_cyber_monday)

# Semana del año, inicio y fin de mes, trimestre
inferencia_df['semana_del_año'] = inferencia_df['fecha'].dt.isocalendar().week
inferencia_df['inicio_mes'] = (inferencia_df['dia_mes'] == 1).astype(int)
inferencia_df['fin_mes'] = (inferencia_df['fecha'] == inferencia_df['fecha'] + pd.offsets.MonthEnd(0)).astype(int)
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter

# Días hasta el siguiente festivo
def dias_hasta_festivo(fecha):
    futuros = [f for f in festivos_ts if f >= fecha]
    if futuros:
        return min([(f - fecha).days for f in futuros])
    return np.nan

inferencia_df['dias_hasta_festivo'] = inferencia_df['fecha'].apply(dias_hasta_festivo)

# Es víspera de festivo
inferencia_df['es_vispera_festivo'] = inferencia_df['fecha'].apply(lambda x: int((x + pd.Timedelta(days=1)) in festivos_ts))

# Mitad de mes (días 13-18)
inferencia_df['es_mitad_mes'] = inferencia_df['dia_mes'].between(13, 18).astype(int)

print('✓ Variables temporales y de calendario creadas')
print(f'✓ Columnas añadidas: año, mes, dia_mes, dia_semana, es_fin_de_semana, es_festivo, es_blackfriday, es_cybermonday, etc.')

✓ Variables temporales y de calendario creadas
✓ Columnas añadidas: año, mes, dia_mes, dia_semana, es_fin_de_semana, es_festivo, es_blackfriday, es_cybermonday, etc.


In [72]:
inferencia_df.shape

(888, 29)

In [73]:
# Paso 3: Variables de Lags y Media Móvil

# Ordenar por producto y fecha
inferencia_df = inferencia_df.sort_values(['producto_id', 'año', 'fecha']).copy()

# Crear lags de 1 a 7 días
for i in range(1, 8):
    inferencia_df[f'lag_{i}'] = inferencia_df.groupby(['producto_id', 'año'])['unidades_vendidas'].shift(i)

# Media móvil de 7 días (desplazada 1 día para evitar data leakage)
inferencia_df['media_movil_7d'] = inferencia_df.groupby(['producto_id', 'año'])['unidades_vendidas'].transform(
    lambda x: x.rolling(7, min_periods=1).mean().shift(1)
)

# Para inferencia, rellenar nulos con 0 en lugar de eliminar filas
# Esto es necesario porque los primeros días no tienen historial suficiente
lag_cols = [f'lag_{i}' for i in range(1, 8)] + ['media_movil_7d']
nulos_antes = inferencia_df[lag_cols].isnull().sum().sum()
inferencia_df[lag_cols] = inferencia_df[lag_cols].fillna(0)

print(f'✓ Variables lag (lag_1 a lag_7) y media_movil_7d creadas')
print(f'✓ Valores nulos rellenados con 0: {nulos_antes}')
print(f'✓ Filas totales conservadas: {len(inferencia_df)}')

✓ Variables lag (lag_1 a lag_7) y media_movil_7d creadas
✓ Valores nulos rellenados con 0: 5616
✓ Filas totales conservadas: 888


In [74]:
inferencia_df.shape

(888, 37)

In [75]:
# Paso 4: Variables de Precios y Competencia

# Variable de descuento porcentaje
inferencia_df['descuento_pct'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100

# Precio promedio de la competencia
compet_cols = ['Amazon', 'Decathlon', 'Deporvillage']
inferencia_df['precio_competencia'] = inferencia_df[compet_cols].mean(axis=1)

# Ratio: nuestro precio vs competencia
inferencia_df['ratio_Precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# Eliminar columnas individuales de competidores
inferencia_df = inferencia_df.drop(columns=compet_cols)

print('✓ Variable descuento_pct creada')
print('✓ Variables precio_competencia y ratio_Precio creadas')
print('✓ Columnas de competidores individuales eliminadas')

✓ Variable descuento_pct creada
✓ Variables precio_competencia y ratio_Precio creadas
✓ Columnas de competidores individuales eliminadas


In [76]:
# Paso 5: One-Hot Encoding de Variables Categóricas

# Guardar información de identificación antes del encoding
inferencia_df['nombre_producto'] = inferencia_df['nombre']
inferencia_df['categoria_producto'] = inferencia_df['categoria']
inferencia_df['subcategoria_producto'] = inferencia_df['subcategoria']

# Aplicar OneHot Encoding
inferencia_df = pd.get_dummies(inferencia_df, columns=['nombre', 'categoria', 'subcategoria'], 
                                prefix=['nombre', 'categoria', 'subcategoria'])

print(f'✓ One-Hot Encoding aplicado')
print(f'✓ Forma del dataframe: {inferencia_df.shape[0]} filas, {inferencia_df.shape[1]} columnas')
print(f'✓ Columnas categóricas codificadas: nombre, categoria, subcategoria')

✓ One-Hot Encoding aplicado
✓ Forma del dataframe: 888 filas, 81 columnas
✓ Columnas categóricas codificadas: nombre, categoria, subcategoria


In [77]:
# Paso 6: Filtrar solo registros de noviembre (eliminar octubre)

filas_antes_filtro = len(inferencia_df)
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()
filas_despues_filtro = len(inferencia_df)

print(f'✓ Filtrado aplicado: solo registros de noviembre')
print(f'✓ Filas eliminadas (octubre): {filas_antes_filtro - filas_despues_filtro}')
print(f'✓ Filas restantes (noviembre): {filas_despues_filtro}')
print(f'✓ Rango de fechas final: {inferencia_df["fecha"].min()} a {inferencia_df["fecha"].max()}')

✓ Filtrado aplicado: solo registros de noviembre
✓ Filas eliminadas (octubre): 168
✓ Filas restantes (noviembre): 720
✓ Rango de fechas final: 2025-11-01 00:00:00 a 2025-11-30 00:00:00


In [78]:
# Paso 7: Guardar el dataframe transformado

output_path = '../data/processed/inferencia_df_transformado.csv'
inferencia_df.to_csv(output_path, index=False)

print(f'✅ Dataframe de inferencia transformado y guardado en: {output_path}')
print(f'📊 Forma del dataframe final: {inferencia_df.shape[0]} filas, {inferencia_df.shape[1]} columnas')

✅ Dataframe de inferencia transformado y guardado en: ../data/processed/inferencia_df_transformado.csv
📊 Forma del dataframe final: 720 filas, 81 columnas


## Visualización del dataframe final transformado

A continuación se muestran las primeras 10 filas del dataframe de inferencia ya transformado y listo para usar con el modelo.

In [79]:
# Visualizar el dataframe final
print('🔍 Información del dataframe transformado:')
print('='*100)
print(f'📌 Total de columnas: {len(inferencia_df.columns)}')
print(f'📌 Total de registros: {len(inferencia_df)}')
print(f'📌 Rango de fechas: {inferencia_df["fecha"].min()} a {inferencia_df["fecha"].max()}')
print(f'📌 Productos únicos: {inferencia_df["producto_id"].nunique()}')
print('='*100)
print('\n🔎 Primeras 10 filas del dataframe:')
inferencia_df.head(10)

🔍 Información del dataframe transformado:
📌 Total de columnas: 81
📌 Total de registros: 720
📌 Rango de fechas: 2025-11-01 00:00:00 a 2025-11-30 00:00:00
📌 Productos únicos: 24

🔎 Primeras 10 filas del dataframe:


,fecha,producto_id,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,año,mes,dia_mes,...,subcategoria_Esterilla Yoga,subcategoria_Mancuernas Ajustables,subcategoria_Mochila Trekking,subcategoria_Pesa Rusa,subcategoria_Pesas Casa,subcategoria_Rodillera Yoga,subcategoria_Ropa Montaña,subcategoria_Ropa Running,subcategoria_Zapatillas Running,subcategoria_Zapatillas Trail
168,2025-11-01,PROD_001,115,True,NaN,115.0,NaN,2025,11,1,...,False,False,False,False,False,False,False,False,True,False
192,2025-11-02,PROD_001,115,True,NaN,115.0,NaN,2025,11,2,...,False,False,False,False,False,False,False,False,True,False
216,2025-11-03,PROD_001,115,True,NaN,115.0,NaN,2025,11,3,...,False,False,False,False,False,False,False,False,True,False
240,2025-11-04,PROD_001,115,True,NaN,115.0,NaN,2025,11,4,...,False,False,False,False,False,False,False,False,True,False
264,2025-11-05,PROD_001,115,True,NaN,115.0,NaN,2025,11,5,...,False,False,False,False,False,False,False,False,True,False
288,2025-11-06,PROD_001,115,True,NaN,115.0,NaN,2025,11,6,...,False,False,False,False,False,False,False,False,True,False
312,2025-11-07,PROD_001,115,True,NaN,115.0,NaN,2025,11,7,...,False,False,False,False,False,False,False,False,True,False
336,2025-11-08,PROD_001,115,True,NaN,115.0,NaN,2025,11,8,...,False,False,False,False,False,False,False,False,True,False
360,2025-11-09,PROD_001,115,True,NaN,115.0,NaN,2025,11,9,...,False,False,False,False,False,False,False,False,True,False
384,2025-11-10,PROD_001,115,True,NaN,115.0,NaN,2025,11,10,...,False,False,False,False,False,False,False,False,True,False


In [80]:
# Verificar las columnas del dataframe
print('📋 Todas las columnas del dataframe transformado:')
print('='*100)
for i, col in enumerate(inferencia_df.columns, 1):
    print(f'{i:3d}. {col}')
print('='*100)

📋 Todas las columnas del dataframe transformado:
  1. fecha
  2. producto_id
  3. precio_base
  4. es_estrella
  5. unidades_vendidas
  6. precio_venta
  7. ingresos
  8. año
  9. mes
 10. dia_mes
 11. dia_semana
 12. nombre_dia
 13. es_fin_de_semana
 14. es_festivo
 15. es_blackfriday
 16. es_cybermonday
 17. semana_del_año
 18. inicio_mes
 19. fin_mes
 20. trimestre
 21. dias_hasta_festivo
 22. es_vispera_festivo
 23. es_mitad_mes
 24. lag_1
 25. lag_2
 26. lag_3
 27. lag_4
 28. lag_5
 29. lag_6
 30. lag_7
 31. media_movil_7d
 32. descuento_pct
 33. precio_competencia
 34. ratio_Precio
 35. nombre_producto
 36. categoria_producto
 37. subcategoria_producto
 38. nombre_Adidas Own The Run Jacket
 39. nombre_Adidas Ultraboost 23
 40. nombre_Asics Gel Nimbus 25
 41. nombre_Bowflex SelectTech 552
 42. nombre_Columbia Silver Ridge
 43. nombre_Decathlon Bandas Elásticas Set
 44. nombre_Domyos BM900
 45. nombre_Domyos Kit Mancuernas 20kg
 46. nombre_Gaiam Premium Yoga Block
 47. nombre_Lifor